In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
def box_score_table_reader(path, start_season, end_season, file_end):
    
    all_df = pd.read_csv(path + 'NBA_' + str(start_season) + file_end)
    all_df['season'] = start_season
    
    for i in range(start_season + 1, end_season):
        df = pd.read_csv(path + 'NBA_' + str(i) + file_end)
        df['season'] = i 
        all_df = pd.concat([all_df, df])
    
    return(all_df)

In [3]:
file_end = '_games.csv'
path = 'C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/'
df = box_score_table_reader(path, 2014, 2022, file_end)

In [4]:
df.head(2)

,Unnamed: 0,starter,Starters,player_id,MP,FG,FGA,FG%,3P,3PA,...,STL,BLK,TOV,PF,PTS,+/-,team,team_id,game_id,season
0,0,1,LeBron James,/players/j/jamesle01.html,38:01,5.0,11.0,0.455,0.0,1.0,...,1.0,0.0,2.0,0.0,17.0,8.0,Miami Heat,/teams/MIA/2014.html,201310290MIA.html,2014
1,1,1,Dwyane Wade,/players/w/wadedw01.html,36:07,5.0,13.0,0.385,1.0,1.0,...,1.0,1.0,3.0,4.0,13.0,2.0,Miami Heat,/teams/MIA/2014.html,201310290MIA.html,2014


In [5]:
df.dtypes

Unnamed: 0      int64
starter         int64
Starters       object
player_id      object
MP             object
FG            float64
FGA           float64
FG%           float64
3P            float64
3PA           float64
3P%           float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
+/-           float64
team           object
team_id        object
game_id        object
season          int64
dtype: object

In [6]:
df.MP = df.MP.replace('Did Not Play', '00:00').replace('Did Not Dress', '00:00').replace('Not With Team', '00:00').replace('Player Suspended', '00:00')

df['played_min'] = df.MP.str.split(':', expand = True)[0]
df['played_sec'] = df.MP.str.split(':', expand = True)[1]

df.MP = df.MP.replace('Did Not Play', np.nan).replace('Did Not Dress', np.nan).replace('Not With Team', np.nan).replace('Player Suspended', np.nan)

df['played'] = np.where(df['FG'].isna(), False, True)
df.MP = np.where(df['MP'] == 'NaT', 0, df['MP'])

In [7]:
df.to_csv('C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/merged_seasons_2014_2021.csv')

In [8]:
file_end = '_gamesbox_score.csv'
path = 'C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/'
df = box_score_table_reader(path, 2014, 2022, file_end)

In [9]:
df['OT_dummy'] = np.where(df.OT.isna(), False, True)
df['hour'] = df['hour'].str.replace('p', '')
df['date'] = pd.to_datetime(df.date, format = '%a, %b %d, %Y')
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
df['day_name'] = df.date.dt.day_name()
df.att = pd.to_numeric(df.att.str.replace(',', ''))

In [10]:
df.dtypes

Unnamed: 0                 int64
date              datetime64[ns]
hour                      object
visitor                   object
visitor_id                object
visitor_score              int64
home                      object
home_id                   object
home_score                 int64
box_score_link            object
OT                        object
att                      float64
season                     int64
OT_dummy                    bool
day                        int64
month                      int64
year                       int64
day_name                  object
dtype: object

Let's convert the databases to the same level and calculate the following features:
- Record/Rank of the team
- Match-up results between the two teams if the teams have already played against one another this season
- Whether the team has played particularly well in the last ten games
- Team statistics such as assists per game, three points shots made per game, defensive blocks per game, et cetera
- Team record at home games, away games, and conference games

In [11]:
box_scores = pd.read_csv('C:/Users/Hp/Desktop/year2021/nba game prediction/nba_data/merged_seasons_2014_2021.csv')

In [12]:
df = df[(df.season == 2019) & (df.date < '2019-01-01')]

In [13]:
df.box_score_link = df.box_score_link.str.rsplit('/').apply(lambda x: x[-1])

In [14]:
box_scores = pd.merge(box_scores, df.loc[:,['date', 'box_score_link']], left_on = 'game_id', right_on = 'box_score_link', how = 'left')

In [15]:
df['home_win'] = np.where(df.home_score > df.visitor_score, 1, 0)
df['visitor_win'] = np.where(df.home_score > df.visitor_score, 0, 1)

In [16]:
df.head(2)

,Unnamed: 0,date,hour,visitor,visitor_id,visitor_score,home,home_id,home_score,box_score_link,OT,att,season,OT_dummy,day,month,year,day_name,home_win,visitor_win
0,0,2018-10-16,10:30,Oklahoma City Thunder,/teams/OKC/2019.html,100,Golden State Warriors,/teams/GSW/2019.html,108,201810160GSW.html,NaN,19596.0,2019,False,16,10,2018,Tuesday,1,0
1,1,2018-10-17,7:00,Milwaukee Bucks,/teams/MIL/2019.html,113,Charlotte Hornets,/teams/CHO/2019.html,112,201810170CHO.html,NaN,17889.0,2019,False,17,10,2018,Wednesday,0,1


In [228]:
def record_calculator (df, game_id):
    
    game_info = df[df.box_score_link == game_id]
    game_date = game_info.date.tolist()[0]
    game_vis = game_info.visitor_id.tolist()[0]
    game_home = game_info.home_id.tolist()[0]
    
    df = df[df.date < game_date]
            
    if df.shape[0] > 0:
            visitor_win_as_visitor = df[(df.visitor_id == game_vis)].visitor_win.sum()
            visitor_win_as_home = df[(df.home_id == game_vis)].home_win.sum()
            visitor_played_games = df[(df.visitor_id == game_vis) | (df.home_id == game_vis)].shape[0]
            
            home_win_as_visitor = df[(df.visitor_id == game_home)].home_win.sum()
            home_win_as_home = df[(df.home_id == game_home)].home_win.sum()
            home_played_games = df[(df.visitor_id == game_home) | (df.home_id == game_home)].shape[0]
            
    else:
        return [0, 0, 0, 0, 0, 0]
    
    return [visitor_win_as_visitor, visitor_win_as_home, visitor_played_games, home_win_as_visitor, home_win_as_home, home_played_games]

In [229]:
def record_tracker(df):
    
    saver = []
    for game_id in df.box_score_link:
        #print(saver)
        saver.append(record_calculator(df, game_id))
    return pd.DataFrame(p, columns = ['v_win_as_v', 'v_win_as_h', 'v_played','h_win_as_v', 'h_win_as_h', 'h_played'])

In [235]:
df = pd.concat([df.reset_index(drop=True), record_tracker(df)], axis=1)

,Unnamed: 0,date,hour,visitor,visitor_id,visitor_score,home,home_id,home_score,box_score_link,...,year,day_name,home_win,visitor_win,v_win_as_v,v_win_as_h,v_played,h_win_as_v,h_win_as_h,h_played
0,0,2018-10-16,10:30,Oklahoma City Thunder,/teams/OKC/2019.html,100,Golden State Warriors,/teams/GSW/2019.html,108,201810160GSW.html,...,2018,Tuesday,1,0,0,0,0,0,0,0
1,1,2018-10-17,7:00,Milwaukee Bucks,/teams/MIL/2019.html,113,Charlotte Hornets,/teams/CHO/2019.html,112,201810170CHO.html,...,2018,Wednesday,0,1,0,0,0,0,0,0
2,2,2018-10-17,7:00,Brooklyn Nets,/teams/BRK/2019.html,100,Detroit Pistons,/teams/DET/2019.html,103,201810170DET.html,...,2018,Wednesday,1,0,0,0,0,0,0,0
3,3,2018-10-17,7:00,Memphis Grizzlies,/teams/MEM/2019.html,83,Indiana Pacers,/teams/IND/2019.html,111,201810170IND.html,...,2018,Wednesday,1,0,0,0,0,0,0,0
4,4,2018-10-17,7:00,Miami Heat,/teams/MIA/2019.html,101,Orlando Magic,/teams/ORL/2019.html,104,201810170ORL.html,...,2018,Wednesday,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,542,2018-12-31,7:00,Memphis Grizzlies,/teams/MEM/2019.html,101,Houston Rockets,/teams/HOU/2019.html,113,201812310HOU.html,...,2018,Monday,1,0,8,10,35,10,12,35
543,543,2018-12-31,7:00,Boston Celtics,/teams/BOS/2019.html,111,San Antonio Spurs,/teams/SAS/2019.html,120,201812310SAS.html,...,2018,Monday,1,0,10,10,34,12,14,37
544,544,2018-12-31,8:00,Minnesota Timberwolves,/teams/MIN/2019.html,114,New Orleans Pelicans,/teams/NOP/2019.html,123,201812310NOP.html,...,2018,Monday,1,0,5,12,36,15,12,37
545,545,2018-12-31,8:00,Dallas Mavericks,/teams/DAL/2019.html,102,Oklahoma City Thunder,/teams/OKC/2019.html,122,201812310OKC.html,...,2018,Monday,1,0,2,15,35,9,12,35


In [374]:
def on_fire_feature(df, n, score_difference, game_id):
    game_info = df[df.box_score_link == game_id]
    game_date = game_info.date.tolist()[0]
    game_vis = game_info.visitor_id.tolist()[0]
    game_home = game_info.home_id.tolist()[0]
    
    df_team1 = df[((df.visitor_id == game_vis) | (df.home_id == game_vis)) & (df.date < game_date)]
    #print(df_team1)
    if df_team1.shape[0] > n:
        team1_visitor = df_team1[df_team1.visitor_id == game_vis]
        team1_visitor['score_diff'] = team1_visitor['visitor_score'] - team1_visitor['home_score']
        team1_home = df_team1[df_team1.home_id == game_vis]
        team1_home['score_diff'] = team1_home['home_score'] - team1_home['visitor_score']
        
        team1 = pd.concat([team1_visitor.reset_index(drop=True), team1_home], axis = 0).sort_values(by = ['date'], ascending=False)
        team1 = team1.head(n)
        
        team1_wins_from_n = sum(1 if x > score_difference else 0 for x in team1.score_diff)
        
    df_team2 = df[((df.visitor_id == game_home) | (df.home_id == game_home)) & (df.date < game_date)]

    if df_team2.shape[0] > n:
        team2_visitor = df_team2[df_team2.visitor_id == game_home]
        team2_visitor['score_diff'] = team2_visitor['visitor_score'] - team2_visitor['home_score']
        team2_home = df_team2[df_team2.home_id == game_home]
        team2_home['score_diff'] = team2_home['home_score'] - team2_home['visitor_score']
        
        team2 = pd.concat([team2_visitor.reset_index(drop=True), team2_home], axis = 0).sort_values(by = ['date'], ascending=False)
        team2 = team2.head(n)

        team2_wins_from_n = sum(1 if x > score_difference else 0 for x in team2.score_diff)    
    
        
        return [team1_wins_from_n, n, team1_wins_from_n/n, team2_wins_from_n, n, team2_wins_from_n/n]

In [375]:
on_fire_feature(df, 5, 0, '201812310PHO.html')

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

[3, 5, 0.6, 1, 5, 0.2]